In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle
import re

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from collections import defaultdict
from pprint import pprint
from config.assistant import *
from config.site_config import *

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
job_group = {}

In [3]:
def save_dictionary(job_company, company_link, job_posting, posting_link):
    global job_group
    
    # 신규 회사명 인입
    if not job_company in job_group:    
        job_group[job_company] = []
    
    for company in job_group[job_company]:
        if company['title'] == job_posting:    # 채용공고명 - 중복
            break
    else:
        job_group[job_company].append({'company': job_company, \
                                       'company_link': company_link, \
                                       'title': job_posting, \
                                       'title_link': posting_link, \
                                       'title_idx': len(job_group[job_company]), \
                                       'input_date': datetime.date.today().isoformat(), \
                                       'status': 'wait'
                                      })
    return None

In [4]:
def filtering_company(job_company):
    company_re = re.search('\(.*\)', job_company)
    if company_re:
        str_filtering = job_company[company_re.span()[0]:company_re.span()[1]]
        job_company = job_company.replace(str_filtering, '')
    job_company = job_company.replace('㈜', '')
    job_company = job_company.replace('㈔', '')
    job_company = job_company.replace('재)', '')
    job_company = job_company.replace('주)', '')
    job_company = job_company.replace(' ', '')
    job_company = job_company.replace('\n', '')
    if job_company in company_filtering_list:
        return 0, "none"

    return 1, job_company


### 사람인

In [6]:
# 사람인
tic()
for search_word in search_words:

    item_count = 0
    current_page = 1
    
    # 검색 결과 리스트 페이지 별 확인
    search_loop = True
    while(search_loop):
        search_link = f'https://www.saramin.co.kr/zf_user/search/recruit'\
            + f'?search_area=main'\
            + f'&search_done=y'\
            + f'&search_optional_item=n'\
            + f'&searchType=search'\
            + f'&recruitSort=relation'\
            + f'&searchword={search_word}'\
            + f'&recruitPage={current_page}'\
            + f'&recruitPageCount={page_view_items}'\
            + f'&company_cd={company_cd}'\
            + f'&mainSearch=y'

        response = requests.get(search_link, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        if len(soup.find_all('div', attrs={'class': 'info_no_result'})) == 1:
            search_loop = False
        else:
            current_page += 1

            page_items = len(soup.find_all('h2', attrs={'class': 'job_tit'}))

            # 해당 페이지 리스트 (1 ~ 100)
            for i in range(1, page_items+1):
                elements = soup.select(f'div.content > div:nth-child({i})')[0]

                # 회사명
                job_company = elements.find('a', attrs={'class': 'track_event data_layer'}).text
                filtering_result = filtering_company(job_company)
                if filtering_result[0] == 0:
                    continue
                else:
                    job_company = filtering_result[1]
                    
                # 채용공고명
                job_posting = elements.find('a', attrs={'class': 'data_layer'})['title']

                # 채용공고 링크
                posting_link = base_link_saram + elements.find('a', attrs={'class': 'data_layer'})['href']

                # 회사 링크
                company_link = base_link_saram + elements.find('a', attrs={'class': 'track_event data_layer'})['href']

                # Dictionary에 저장
                save_dictionary(job_company, company_link, job_posting, posting_link)
                
                item_count += 1
                
    print(f'{search_word}: {item_count}')
toc()

컴퓨터비전: 1208
COMPUTER VISION: 1052
영상처리: 1079
IMAGE PROCESSING: 291
딥러닝: 1188
DEEP LEARNING: 1188
머신러닝: 1507
MACHINE LEARNING: 1500
이미지인식: 211
IMAGE RECOGNITION: 21
이미지분석: 705
IMAGE ANALYSIS: 29
VISION AI: 777
OPENCV: 247
Elapsed time is 603.0929794311523 seconds.


In [7]:
len(job_group)

4915

In [8]:
nCnt = 0
for i in job_group:
    for j in job_group[i]:
        nCnt += 1
print(nCnt)

10015


### 잡코리아

In [10]:
# 잡코리아
# 저장된 키워드 별 검색

tic()
for search_word in search_words:

    item_count = 0
    current_page = 1
    
    # 검색 결과 리스트 페이지 별 확인
    search_loop = True
    while(search_loop):
        search_link = f'https://www.jobkorea.co.kr/Search/' \
                    +f'?stext={search_word}' \
                    +f'&tabType=recruit&Page_No={current_page}'

        response = requests.get(search_link, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        page_items = len(soup.find_all('a', attrs={'class': 'title dev_view'}))
        
        if page_items == 0:
            search_loop = False
        else:
            for i in range(0, page_items):
                # 회사명
                job_company = soup.find_all('a', attrs={'class': 'name dev_view'})[i].text
                filtering_result = filtering_company(job_company)
                if filtering_result[0] == 0:
                    continue
                else:
                    job_company = filtering_result[1]
                
                # 회사 링크
                company_link = base_link_korea + soup.find_all('a', attrs={'class': 'name dev_view'})[i]['href']
                
                # 채용공고
                job_posting = soup.find_all('a', attrs={'class': 'title dev_view'})[i].text
                job_posting = job_posting.replace('\r\n', '')
                job_posting = job_posting.strip()

                # 채용공고 링크
                posting_link = base_link_korea + soup.find_all('a', attrs={'class': 'title dev_view'})[i]['href']

                # Dictionary에 저장
                save_dictionary(job_company, company_link, job_posting, posting_link)
                
                item_count += 1
        current_page += 1
    print(f'{search_word}: {item_count}')
toc()

컴퓨터비전: 1165
COMPUTER VISION: 117
영상처리: 1171
IMAGE PROCESSING: 60
딥러닝: 1075
DEEP LEARNING: 181
머신러닝: 1747
MACHINE LEARNING: 1749
이미지인식: 125
IMAGE RECOGNITION: 27
이미지분석: 2312
IMAGE ANALYSIS: 28
VISION AI: 252
OPENCV: 149
Elapsed time is 556.5792679786682 seconds.


In [11]:
len(job_group)

5146

In [12]:
nCnt = 0
for i in job_group:
    for j in job_group[i]:
        nCnt += 1
print(nCnt)

10765


In [13]:
# JSON 파일 쓰기
tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

In [5]:
# JSON 파일 불러오기
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

4674
